In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
# Use in case of lack packages
!pip install -U gensim==3.6.0 pyunpack tqdm sklearn patool requests pandas flask

Requirement already up-to-date: gensim==3.6.0 in /usr/local/lib/python3.5/dist-packages (3.6.0)
Requirement already up-to-date: pyunpack in /usr/local/lib/python3.5/dist-packages (0.1.2)
Requirement already up-to-date: tqdm in /usr/local/lib/python3.5/dist-packages (4.29.1)
Requirement already up-to-date: sklearn in /usr/local/lib/python3.5/dist-packages (0.0)
Requirement already up-to-date: patool in /usr/local/lib/python3.5/dist-packages (1.12)
Requirement already up-to-date: requests in /usr/local/lib/python3.5/dist-packages (2.21.0)
Requirement already up-to-date: pandas in /usr/local/lib/python3.5/dist-packages (0.23.4)
    100% |████████████████████████████████| 92kB 626kB/s ta 0:00:01
    100% |████████████████████████████████| 327kB 2.0MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 2.1MB/s ta 0:00:011


In [6]:
from tqdm.autonotebook import tqdm
from wsd import *
from parsers import *
import pandas as pd

import plwn

ValueError: emb_ws/embeddings/fastText-plWNC-skipgram-300-lemmas-mwe-minCount-50.bin cannot be opened for loading!

In [12]:
remote_embeddings_address = 'http://10.17.5.15:4000'
stop_words_file = 'polish.stopwords.txt'

# we = RemoteWordEmbedding(remote_embeddings_address)
# se = RemoteSenseEmbedding(remote_embeddings_address)
we = get_ft_word_embeddings()
se = get_sense_embeddings()

# we = LocalEmb(get_we)
# se = LocalEmb(get_se)

sw = StopWord(stop_words_file)
wn = Wordnet(plwn.load_default())

skladnica = load_skladnica(WSDParseType.wsd)
kpwr = load_kpwr()

lemmas = create_lemmas(*skladnica[:,1], *kpwr[:,1], sw=sw)

wsd = WSD(we, se, sw, wn, lemmas, tqdm_disable=False)

Loading skladnica...
Loading kpwr...


In [6]:
import warnings
from sklearn.exceptions import DataConversionWarning, UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)
warnings.filterwarnings(action='ignore', category=RuntimeWarning)
                        
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from functools import partial
from itertools import product, starmap
import pandas as pd
import os

def score(idx, sentence, senses, use_related, write_to_scores, get_results_by_idx):
    results_from_file = get_results_by_idx(idx)
    if results_from_file is not None:
        return results_from_file
    
    try:
        wsd_out = wsd(sentence, use_related=use_related)
    except (ValueError, KeyError) as e:
        print('FAILED', idx, sentence)
        print('ERROR:', e)
        return [[[np.NaN,] * 4, ] * 3] * 2
    
    def _scores(true, preds):
        acc = accuracy_score(true, preds)
        f1 = f1_score(true, preds, average='macro')
        prec = precision_score(true, preds, average='macro')
        rec = recall_score(true, preds,average='macro')
        return acc, f1, prec, rec
    
    real_senses = list(filter(lambda word_sense: word_sense[1], senses))
    
    filter_type = lambda type_name: list(filter(lambda word_sense: word_sense[2] == type_name, real_senses))
    create_type_dict = lambda data: {elem[0]: elem[1:] for elem in data}
    
    real_senses_verbs = create_type_dict(filter_type('verb'))
    real_senses_nouns = create_type_dict(filter_type('noun'))
    real_senses_others = create_type_dict(filter_type('other'))

    # mati version
    def mati_score(true_vals):
        true, preds = [], []
        for word, sens_id in wsd_out.items():
            true.append(sens_id[1])
            pred = true_vals.get(word)
            preds.append(int(pred[0]) if pred else 0)
        
        return _scores(true, preds)
    
    # grzech version
    def grzech_score(true_vals):
        true, preds = [], []
        for word, (sens_id, _) in true_vals.items():
            true.append(int(sens_id))
            pred = wsd_out.get(word)
            preds.append(pred[1] if pred else 0)
        
        return _scores(true, preds)
    
    real_senses = [real_senses_verbs, real_senses_nouns, real_senses_others]
    
    mati_verses = list(map(mati_score, real_senses)) 
    grzech_verses = list(map(grzech_score, real_senses))
    
    write_to_scores(','.join([str(idx), 
                              *map(str, list(chain(*mati_verses))), 
                              *map(str, list(chain(*grzech_verses)))]))
    
    return np.asarray([mati_verses, grzech_verses])

In [7]:
def test(dataset_type: WSDParseType, use_related):
    if dataset_type == WSDParseType.kpwr:
        dataset = kpwr
        dataset_name = 'kpwr'
    else:
        dataset = skladnica
        dataset_name = 'skladnica'
        
    related_name = '_related' if use_related else ''
    
    scores_filename = 'scores_{}{}.csv'.format(dataset_name, related_name)

    def write_to_scores(text, filename=scores_filename):
        with open(filename, 'a+') as f:
            f.write(text + '\n')
        
    def get_results_by_idx(idx, filename=scores_filename):
        with open(filename, 'r') as f:
            for line in f.read().strip().split('\n'):
                scores = line.split(',')
                fidx = int(scores[0]) if scores[0] != 'idx' else -1
                if fidx == idx:
                    print('Found', idx, 'sentence', end='\r')
                    scores = list(map(float, scores[1:]))
                    mati_scores_verb = scores[:4]
                    mati_scores_noun = scores[4:8]
                    mati_scores_other = scores[8:12]
                    grzech_scores_verb = scores[12:16]
                    grzech_scores_noun = scores[16:20]
                    grzech_scores_other = scores[20:]
                    
                    mati_scores = [mati_scores_verb, mati_scores_noun, mati_scores_other]
                    grzech_scores = [grzech_scores_verb, grzech_scores_noun, grzech_scores_other]
                    scores = np.asarray([mati_scores, grzech_scores])
                    assert len(mati_scores_verb) == len(mati_scores_noun) == len(mati_scores_other) == len(grzech_scores_verb) == len(grzech_scores_noun) == len(grzech_scores_other)                    
                    return scores
            return None
    
    if not os.path.isfile(filename):
        write_to_scores('idx,' + ','.join(['_'.join(data) for data in product(['mati', 'grzech'], 
                                                                          ['verb', 'noun', 'other'],
                                                                          ['acc', 'f1', 'prec', 'rec'], )]))
    
    _score = partial(score, 
                     use_related=True, 
                     write_to_scores=write_to_scores,
                     get_results_by_idx=get_results_by_idx)
    
    ress = list(starmap(_score, tqdm(dataset[:1000], 'Sentence', dynamic_ncols=True)))

    def print_scores(results):
           
        results = np.array(results)

        mati_vers, grzech_vers = results[:,0,:], results[:,1,:]

        def filter_nans(vers_results):
            return np.asarray(list(filter(lambda x: any(not np.isnan(z) for y in x for z in y), vers_results)))
    
        def _print_scores(version: str, results, pos_type):
            if pos_type == 'verb':
                pos_pos = 0
            elif pos_type == 'noun':
                pos_pos = 1
            else:
                pos_pos = 2
                
            print(pos_type)
            print('\tacc ', version, results[pos_pos][0])
            print('\tf1  ', version, results[pos_pos][1])
            print('\tprec', version, results[pos_pos][2])
            print('\trec ', version, results[pos_pos][3])
        
        mati_vers = filter_nans(mati_vers)
        grzech_vers = filter_nans(grzech_vers)

        mati_vers = np.average(mati_vers, axis=0)
        grzech_vers = np.average(grzech_vers, axis=0)
        
        
        _print_scores('mati  \t', mati_vers, 'verb')
        _print_scores('mati  \t', mati_vers, 'noun')
        _print_scores('mati  \t', mati_vers, 'other')
        
        _print_scores('grzech\t', grzech_vers, 'verb')
        _print_scores('grzech\t', grzech_vers, 'noun')
        _print_scores('grzech\t', grzech_vers, 'other')
        
    print_scores(ress)

In [133]:
%%time
ress = test(WSDParseType.wsd, False)

FAILED 299 Jest potwierdzeniem dla bliźniego życia wartości .
ERROR: 'all ngrams for word · absent from model'


FAILED 300 Ono wartość . posiada
ERROR: 'all ngrams for word · absent from model'


FAILED 304 Ponadto zabrali mu zegarek o wartości 260 złotych . napastnicy
ERROR: 'all ngrams for word · absent from model'


FAILED 846 Jednak temperatura ta jest niższa od wartości otrzymanych w pomiarach .
ERROR: 'all ngrams for word · absent from model'
verbd 999 sentence
	acc  mati  	 0.09490219030104428
	f1   mati  	 0.07486457023131177
	prec mati  	 0.07486457023131177
	rec  mati  	 0.07486457023131177
noun
	acc  mati  	 0.24042697032453958
	f1   mati  	 0.18631417503393044
	prec mati  	 0.18639549535601221
	rec  mati  	 0.18630151039360623
other
	acc  mati  	 0.04774319028650107
	f1   mati  	 0.038544014775817696
	prec mati  	 0.038544014775817696
	rec  mati  	 0.038544014775817696
verb
	acc  grzech	 nan
	f1   grzech	 nan
	prec grzech	 nan
	rec  grzech	 nan
noun
	acc  grzech	 nan
	f1   grzech	 nan
	prec grzech	 nan
	rec  grzech	 nan
other
	acc  grzech	 nan
	f1   grzech	 nan
	prec grzech	 nan
	rec  grzech	 nan
CPU times: user 4.36 s, sys: 2.86 s, total: 7.22 s
Wall time: 13.7 s


In [134]:
%%time
ress = test(WSDParseType.wsd, True)

FAILED 299 Jest potwierdzeniem dla bliźniego życia wartości .
ERROR: 'all ngrams for word · absent from model'


FAILED 300 Ono wartość . posiada
ERROR: 'all ngrams for word · absent from model'


FAILED 304 Ponadto zabrali mu zegarek o wartości 260 złotych . napastnicy
ERROR: 'all ngrams for word · absent from model'


FAILED 846 Jednak temperatura ta jest niższa od wartości otrzymanych w pomiarach .
ERROR: 'all ngrams for word · absent from model'


KeyboardInterrupt: 

In [135]:
%%time
ress = test(WSDParseType.kpwr, False)

KeyboardInterrupt: 

In [136]:
%%time
ress = test(WSDParseType.kpwr, False)

KeyboardInterrupt: 

In [13]:
# text = 'zamek królewski'
# text = 'zamek w spodniach'
# text = 'zamek w pistolecie'
text = 'zamek w drzwiach od domu'
# text = 'zamek króla'

wsd_out = wsd(text, use_related=True)

for word, word_out in wsd_out.items():
    lemma, sense, _ = word_out
    print('*', word, ':', get_senses(lemma, wn).get(sense))

* zamek : mechanizm lub urządzenie do zamykania drzwi, szuflad, walizek.
* domu : budynek przeznaczony na mieszkania, także oficyna.
* drzwiach : ruchoma przegroda (zazwyczaj pionowa), element zamykający otwór w ścianie budynku, ściance mebli, karoserii pojazdu wraz z konstrukcją niezbędną do umocowania tego elementu.


In [ ]:
show_scores = lambda filename: pd.DataFrame(pd.read_csv(f'scores_{filename}.csv').mean()).transpose()

In [ ]:
show_scores('skladnica')

In [ ]:
show_scores('skladnica_related')

In [ ]:
show_scores('kpwr')

In [ ]:
show_scores('kpwr_related')

In [3]:
import 